In [1]:
import numpy as np
import math

# Problem 5

Projections from problem 3.:

In [2]:
def projectionA(X):
    x, y = X
    if x**2 + y**2 <= 1.5:
        return x, y
    else:
        return np.sqrt(1.5 / (x**2 + y**2)) * np.array([x, y])


def projectionB(X):
    x, y = X
    return np.array([projectionB1(x), projectionB1(y)])


def projectionB1(x):
    if x < -1:
        return -1
    elif x > 1:
        return 1
    else:
        return x


def projectionC(X):
    x, y = X
    if -1 <= x <= 1.5 and -1 <= y <= 1.5 and y <= 0.5 - x:
        return np.array([x, y])
    if x < -1:
        if y < -1:
            return np.array([-1, -1])
        elif y < 1.5:
            return np.array([-1, y])
        else:
            return np.array([-1, 1.5])
    elif y < -1:
        if x < 1.5:
            return np.array([x, -1])
        else:
            return np.array([1.5, -1])
    elif y > 2.5 + x:
        return np.array([-1, 1.5])
    elif x > 2.5 + y:
        return np.array([1.5, -1])
    else:
        return np.array([(0.5 - y + x) / 2, (0.5 - x + y) / 2])


Function from problem 2. and its gradient, projected gradient descent to find its minimum:

In [3]:
def f(X):
    x, y = X
    return x**2 + np.exp(x) + y**2 - x*y


def gradf(X):
    x, y = X
    return np.array([2*x + np.exp(x) - y, 2*y - x])


def proj_gd_step(X, projection, learning_rate):
    # print(f'point: {X - learning_rate*(gradf(X))}')
    # print(projection(X - learning_rate*(gradf(X))))
    return projection(X - learning_rate*(gradf(X)))


Setting the parameters for gradient descents:

In [4]:
X_1 = np.array([-1, 1])
actual_min = np.array([-0.43256275, -0.43256275/2])

L = np.sqrt(72 + 12*np.exp(2) + np.exp(4))
beta = (4 + np.exp(2) + np.sqrt(4 + np.exp(4))) / 2
alpha = (4 + np.exp(-2) - np.sqrt(4 + np.exp(-4))) / 2

T = 11                                                              # we are doing 10 steps -> we have 11 points
dist_x1_act = np.linalg.norm(X_1 - actual_min)

learning_rates = [dist_x1_act / (L*np.sqrt(T)), 1/beta, 2/alpha]    # 1., 2. and 3., 4. inequality
projections = [projectionA, projectionB, projectionC]               # circle, square, triangle

In [5]:
learning_rates

[0.027581075359712888, 0.1050199595887474, 1.877263037818603]

In 4-dim array ```points``` we will save all the points from gradient descends with different learning rates and domains for projection.

In [6]:
points = np.zeros([3, 3, 11, 2])     # projections, learning_rates, iterations, x/y
points[:, :, 0, :] = X_1

for p in range(len(projections)):
    for l in range(len(learning_rates)):
        for k in range(1, T):
            lr = learning_rates[l]
            if l == 2:
                lr = lr / (k+1)
            points[p, l, k, :] = proj_gd_step(points[p, l, k-1, :], projections[p], lr)

In [7]:
np.linalg.norm(points[:, 2, -1, :] - actual_min, axis=1)

array([0.00257168, 0.00831117, 0.01194306])

Comparisons from Theorem 3.3. In 3 dimensional array ```ineq``` we have 4 inequalities (first index) for projections on 3 different domains (second index). In third dimension we saved the values of left (0) and right (1) side of inequality. The right side is the same for all the projections, since it is only dependand on starting parameters.

In [10]:
ineq = np.zeros([4, 3, 2])          # inequality, domain, left/right side
for p in range(len(projections)):
    ineq[0, p, :] = [f(sum(points[p, 0, :, :])/T) - f(actual_min), L*dist_x1_act/np.sqrt(T)]
    ineq[1, p, :] = [f(points[p, 1, -1, :]) - f(actual_min), (3*beta*dist_x1_act**2 + f(X_1) - f(actual_min)) / T]
    kappa = beta/alpha
    ineq[2, p, :] = [f(points[p, 1, -1, :]) - f(actual_min), beta/2 * ((kappa-1)/kappa)**(2*(T-1))*dist_x1_act**2]
    ineq[3, p, :] = [f(2/(T*(T+1))*np.matmul(np.array(range(1, 12)), points[p, 2, :, :])) - f(actual_min),
                    2*L**2/(alpha*(T+1))]

In [11]:
ineq

array([[[1.03227949e+00, 5.93729047e+00],
        [1.15791251e+00, 5.93729047e+00],
        [1.15791251e+00, 5.93729047e+00]],

       [[1.51814100e-02, 4.91230558e+00],
        [1.51814100e-02, 4.91230558e+00],
        [1.51814100e-02, 4.91230558e+00]],

       [[1.51814100e-02, 7.99212423e-01],
        [1.51814100e-02, 7.99212423e-01],
        [1.51814100e-02, 7.99212423e-01]],

       [[6.25513352e-05, 3.36760375e+01],
        [3.36122482e-03, 3.36760375e+01],
        [7.30601333e-03, 3.36760375e+01]]])

#### First inequality
The upper bound here is equal to *5.93729047*, below are the values of the left side for each domain:

In [12]:
print(f'Circle: {ineq[0,0,0]}, square: {ineq[0,1,0]}, triangle: {ineq[0,2,0]}')

Circle: 1.0322794917332974, square: 1.1579125074614298, triangle: 1.1579125074614298


#### Second inequality
Here we took ```k=11```, to compare the value of f at the point from last step of our gradient descent with value of f at real minimum.
The upper bound here is equal to *4.91230558*, below are the values of the left side for each domain:

In [13]:
print(f'Circle: {ineq[1,0,0]}, square: {ineq[1,1,0]}, triangle: {ineq[1,2,0]}')

Circle: 0.015181409998737405, square: 0.015181409998737405, triangle: 0.015181409998737405


#### Third inequality
In this inequality we took ```k=10```, to again compare value of f in the last point we calculated with 10 steps (so these values are the same as in second inequality).
The upper bound here is equal to *0.799212423*, below are the values of the left side for each domain:

In [14]:
print(f'Circle: {ineq[2,0,0]}, square: {ineq[2,1,0]}, triangle: {ineq[2,2,0]}')

Circle: 0.015181409998737405, square: 0.015181409998737405, triangle: 0.015181409998737405


#### Fourth inequality

The upper bound here is equal to *33.6760375*, below are the values of the left side for each domain:

In [15]:
print(f'Circle: {ineq[3,0,0]}, square: {ineq[3,1,0]}, triangle: {ineq[3,2,0]}')

Circle: 6.255133523047629e-05, square: 0.003361224817534092, triangle: 0.007306013330456884


We can observe that all of the guarantees hold, but some of them are more precise than others. Since second and third are guarantees for the same value, we can conclude that the third one is much better for this example, since it is more precise.

#### Comparing the results
We will look at how far the final points of gradient descend are from the actual minimum.

In [16]:
np.linalg.norm(points[:, :, -1, :] - actual_min, axis=2)

array([[0.6049279 , 0.15288255, 0.00257168],
       [0.63381875, 0.15288255, 0.00831117],
       [0.63381875, 0.15288255, 0.01194306]])

Above we can see the results for different learning rates (columns) when we are projecting on different sets (rows: circle, square, triangle). We can see that the best learning rate in all cases is the last one, which is changing in each iteration: 2/(alpha* (k+1)).

# Problem 6

In [18]:
a = np.array([[3, 10, 30],[0.1, 10, 35],[3, 10, 30],[0.1, 10, 35]])
c = np.array([1, 1.2, 3, 3.2])
p = np.array([[0.36890, 0.1170, 0.2673], [0.46990, 0.4387, 0.7470], [0.10910, 0.8732, 0.5547], [0.03815, 0.5743, 0.8828]])

In [19]:
def f6(z):
    result = 0
    for i in range(4):
        ex = 0
        for j in range(3):
            ex -= a[i, j] * (z[j] - p[i, j])**2
        result -= c[i] * np.exp(ex)
    return result
        
def gradf6(z):
    result = [0, 0, 0]
    for i in range(4):
        ex = 0
        for j in range(3):
            ex -= a[i, j] * (z[j] - p[i, j])**2
        for k in range(3):
            result[k] += c[i] * np.exp(ex) * 2 * a[i, k] * (z[k] - p[i, k])
    return np.array(result)

def p_GD_step(X, learning_rate, projection=lambda x: x):
    return projection(X - learning_rate*(gradf6(X)))

def p_GD(X, n_steps, learning_rate, projection=lambda x: x):
    points = [X]
    for i in range(n_steps):
        points.append(p_GD_step(points[-1], learning_rate, projection))
    return points

In [20]:
gradf6([1,2,3])

array([6.10421470e-78, 9.04795039e-76, 4.70275110e-75])

In [21]:
f6(p_GD(np.array([1,1,1]), 1000, 0.01)[-1])

-3.8627821477879825

In [22]:
start = [1,1,1]
steps = 0
minimal_value = -3.86278214782076
lr = 0.01
x = start
while f6(x) - minimal_value > 1e-14:
    x = p_GD_step(x, lr)
    steps += 1

print(f'Reached 1e-14 in {steps} steps.')
while f6(x) - minimal_value > 0.5e-14:
    x = p_GD_step(x, lr/100)
    steps += 1
print(f'Reached 0.5e-14 in {steps} steps.')
while f6(x) - minimal_value > 0.4e-14:
    x = p_GD_step(x, lr/100)
    steps += 1
print(f'Reached 0.4e-14 in {steps} steps.')

Reached 1e-14 in 1359 steps.
Reached 0.5e-14 in 8373 steps.
Reached 0.4e-14 in 20847 steps.


In [8]:
print(f'With learning rate 0.01 we reach smaller difference than 1e-14: {abs(f6(x) - minimal_value)} in {steps} steps.')

With learning rate 0.01 we reach smaller difference than 1e-14: 9.769962616701378e-15 in 1359 steps.


In [10]:
print(f'With learning rate 0.001 we reach smaller difference than 1e-14: {abs(f6(x) - minimal_value)} in {steps} steps')

With learning rate 0.001 we reach smaller difference than 1e-14: 9.769962616701378e-15 in 13638 steps


In [12]:
print(f'With learning rate 0.0001 we reach smaller difference than 1e-14: {abs(f6(x) - minimal_value)} in {steps} steps')

With learning rate 0.0001 we reach smaller difference than 1e-14: 9.769962616701378e-15 in 136294 steps


In [12]:
print(f'With learning rate 0.01 for first 1359 and then 0.0001, we reach difference of {abs(f6(x) - minimal_value)} in {steps} steps')

With learning rate 0.01 for first 1359 and then 0.0001, we reach difference of 3.9968028886505635e-15 in 20847 steps


In [24]:
print(f'Value of f6(x): {f6(x)}')

Value of f6(x): -3.862782147820756


We can see that now we have reached the value that rounds to given minimal value of f.